In [22]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:90%;}
div.CodeMirror {font-family:Consolas; font-size:18pt;}
div.output {font-size:19pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

In [2]:
# GPU 사용 여부 확인
import tensorflow as tf
print(tf.__version__)
tf.config.list_physical_devices('GPU')

2.10.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## 머신러닝/딥러닝 : 데이터의 법칙성을 추출
    - 다양한 분야에서 응용 : 글씨인식, 문장분류, 질병진단, 얼굴식별, etc...
    
    - 지도학습(회귀, 분류) vs 비지도학습(군집) vs 강화학습(게임)
    
- 1. 데이터 확보 및 생성(inputdata, outputdata)
- 2. 데이터 전처리 (결측치, 스케일조정, 훈련데이터셋과 검증데이터셋, 시험데이터셋으로 분류....)
- 3. 모델 구성
- 4. 모델 학습 과정 설정(손실함수, 옵티마이저, 평가지표)
- 5. 모델 학습시키기
- 6. 모델 평가(시험데이터셋으로)
- 7. 모델 사용(모델 저장 후, 저장된 모델을 이용하여, 입력값이 주어지면 예측값 받기)

# 1. 데이터 셋 생성
# 2. 데이터 전처리

In [3]:
import numpy as np
from tensorflow.keras.utils import to_categorical # 분류분석시 원핫인코딩
from tensorflow.keras.models import Sequential  # 모델 생성
from tensorflow.keras.layers import Dense, Input # 모델에 레이어 추가시 

In [4]:
# 학습데이터 == 훈련데이터
X_train = np.array([1,2,3,4,5,6,7,8,9]*10)
Y_train = np.array([2,4,6,8,10,12,14,16,18]*10)
# 검증데이터
X_val = np.array([1,2,3,4,5,6,7,8,9])
Y_val = np.array([2,4,6,8,10,12,14,16,18])

In [13]:
# 분류분석을 위한 인코딩 종류
data = np.array(['a','b','b','b','c'])
print('1. 원 데이터 :', data)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labeling_data = le.fit_transform(data)
print('2. 라벨인코딩된 데이터 :', labeling_data)
one_hot_encoding_data = to_categorical(labeling_data)
print('3. 원핫인코딩된 데이터(keras) :\n', one_hot_encoding_data)

1. 원 데이터 : ['a' 'b' 'b' 'b' 'c']
2. 라벨인코딩된 데이터 : [0 1 1 1 2]
3. 원핫인코딩된 데이터(keras) :
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [23]:
# 분류분석을 위한 원핫인코딩 방법
data = np.array([1,2,2,5,9])
print('원 데이터 : ', data)
categorical_one_hot = to_categorical(data)
print('to_categorical을 이용한 원 핫 인코딩(넘파이배열)\n', categorical_one_hot)
import pandas as pd
display('get_dummies를 이용한 원핫인코딩(데이터프레임)\n', pd.get_dummies(data))

원 데이터 :  [1 2 2 5 9]
to_categorical을 이용한 원 핫 인코딩(넘파이배열)
 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


'get_dummies를 이용한 원핫인코딩(데이터프레임)\n'

,1,2,5,9
0,1,0,0,0
1,0,1,0,0
2,0,1,0,0
3,0,0,1,0
4,0,0,0,1


In [27]:
Y_train[:2], Y_val[:2]

(array([2, 4]), array([2, 4]))

In [28]:
# 분류분석을 위해 종속변수(target변수)를 원핫인코딩
Y_train = to_categorical(Y_train, 19)
Y_val   = to_categorical(Y_val)

In [29]:
Y_train[:2]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]], dtype=float32)

In [32]:
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape

((90,), (90, 19), (9,), (9, 19))

In [33]:
X_train[:2]

array([1, 2])

In [34]:
Y_train[:2]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]], dtype=float32)

# 3. 모델 구성

In [35]:
model = Sequential()
model.add(Input(shape=(1,)))
model.add(Dense(units=38, activation='sigmoid')) # units:출력수, activation(활성화함수)
model.add(Dense(units=64, activation="elu"))
model.add(Dense(units=32, activation="elu"))
model.add(Dense(units=19, activation='softmax'))
# activation='softmax' : 모든 출력의 합이 1. 분류분석의 마지막 layer에서 activation함수로 사용
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 38)                76        
                                                                 
 dense_1 (Dense)             (None, 64)                2496      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 19)                627       
                                                                 
Total params: 5,279
Trainable params: 5,279
Non-trainable params: 0
_________________________________________________________________
None


# 4. 모델 학습과정 설정
- 손실함수, 옵티마이저, 평가지표
    * loss='categorical_crossentropy' : 딥러닝 훈련시 다중분류분석에서의 손실함수
            - 모델의 예측 확률분포와 실제값의 확률 분포가 얼마나 다른지 계산

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])